In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# Grid setup
x_lims = [0, 5]
y_lims = [0, 6]
x_array = np.arange(x_lims[0], x_lims[1], 0.3)
y_array = np.arange(y_lims[0], y_lims[1], 0.3)
xx, yy = np.meshgrid(x_array, y_array)

# Vector field definition
f_x = lambda x, y: x ** 2 * y
f_y = lambda x, y: y - 3
M = f_x(xx, yy)
N = f_y(xx, yy)

# Magnitude calculation and normalization
magnitude_F = np.sqrt(M**2 + N**2)
copy_magnitude_F = np.copy(magnitude_F)
magnitude_F[magnitude_F == 0] = np.nan
M_unit = M / magnitude_F
N_unit = N / magnitude_F

# Flatten arrays
X = xx.flatten()
Y = yy.flatten()
U = M_unit.flatten()
V = N_unit.flatten()
Mag = copy_magnitude_F.flatten()

# Create colorscale based on magnitude
colors = px.colors.sample_colorscale("Viridis", (Mag - np.nanmin(Mag)) / (np.nanmax(Mag) - np.nanmin(Mag)))

# Create figure and manually draw arrows
fig = go.Figure()

for i in range(len(X)):
    if np.isnan(Mag[i]):
        continue
    fig.add_trace(
        go.Scatter(
            x=[X[i], X[i] + U[i] * 0.2],
            y=[Y[i], Y[i] + V[i] * 0.2],
            line=dict(color=colors[i], width=2),
            showlegend=False,
            hoverinfo="skip",
            marker=dict(size=10, symbol="arrow-bar-up", angleref="previous"),
        )
    )

# Add closed path rectangle with arrows
path_x_vertices = [1, 4, 4, 1, 1]
path_y_vertices = [1, 1, 5, 5, 1]
names = ["", "Path C_1", "Path C_2", "Path C_3", "Path C_4"]
for idx in range(1, len(path_x_vertices)):
    fig.add_trace(
        go.Scatter(
            x=np.linspace(path_x_vertices[idx - 1], path_x_vertices[idx], 10),
            y=np.linspace(path_y_vertices[idx - 1], path_y_vertices[idx], 10),
            mode="lines+markers",
            marker=dict(size=10, symbol="arrow-bar-up", angleref="previous"),
            line=dict(color="red", width=3, dash="dash"),
            name=names[idx],
        )
    )

# Add axes
fig.add_hline(y=0)
fig.add_vline(x=0)

# Layout
fig.update_layout(
    title="Colored Vector Field (Normalized with Magnitude Coloring)",
    xaxis_title="x-axis",
    yaxis_title="y-axis",
    height=600,
    width=1.6 * 600,
    xaxis_range=[x_lims[0], x_lims[1]],
    yaxis_range=[y_lims[0], y_lims[1]],
    coloraxis_colorbar=dict(title="Magnitude")
)

fig.add_trace(
    go.Scatter(
        x=[None],
        y=[None],
        mode="markers",
        marker=dict(
            colorscale="Viridis",
            cmin=np.nanmin(Mag),
            cmax=np.nanmax(Mag),
            color=[np.nanmin(Mag)],
            colorbar=dict(
                title="Magnitude",
                x=1.02,
                y=0.4,
                xanchor='left',
                yanchor='middle',
                len=0.75
            ),
            showscale=True,
        ),
        hoverinfo="skip",
        showlegend=False
    )
)

fig.show()